<a href="https://colab.research.google.com/github/kimsojeong1225/DeepLearning/blob/master/BostonHousing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/kimsojeong1225/DeepLearning.git

Cloning into 'DeepLearning'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


## 텐서보드 설정

In [0]:
pip install tensorboardcolab

In [0]:
 ;; 67e
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

Using TensorFlow backend.


In [0]:

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-12-31 03:23:52--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.193.139.214, 3.225.9.174, 52.204.223.154, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.193.139.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  17.4MB/s    in 0.8s    

2019-12-31 03:23:58 (17.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
from  datetime import datetime
timestemp = datetime.now()

LOG_DIR = './log/' + str(timestemp) # 텐서보드 로그 경로가 './log'인 경우
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &' # 포트 번호는 6006
    .format('./log')
)


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://1b3102c6.ngrok.io


In [0]:
LOG_DIR

'./log/2019-12-31 06:23:03.483279'

In [0]:
tb_callback = TensorBoard(log_dir='./log',
                          histogram_freq=1,
                          write_graph=True,
                          write_grads=True,
                          batch_size=1,
                          write_images=True)

## 보스턴 집값 데이터 받아오기

In [0]:
from keras.datasets import boston_housing
(train_data,train_targets),(test_data,test_targets)=boston_housing.load_data()

In [0]:
train_data.shape

(404, 13)

In [0]:
test_data.shape

(102, 13)

In [0]:
train_targets[:10]

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4])

In [0]:
# Standardization
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
x_train,x_valid,y_train,y_valid=train_test_split(train_data,train_targets)

##모델 만들기 , 학습

In [0]:
#wide and deep network 함수형 모델
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras import Model


input_tensor = Input(shape=(13,))
x = layers.Dense(13, activation='relu')(input_tensor)
x = layers.Dense(26, activation='relu')(x)
x  = layers.Dense(13, activation='relu')(x)
concat = layers.concatenate( [ x, input_tensor ] )
output_tensor = layers.Dense(1)(concat)
model= Model(input_tensor, output_tensor)
model.compile(loss='mae', optimizer='rmsprop', metrics=['mae'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 13)           182         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 26)           364         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 13)           351         dense_1[0][0]                    
______________________________________________________________________________________________

In [0]:
history=model.fit(x_train,y_train,epochs=20,validation_data=(x_valid,y_valid),batch_size=1,
                  callbacks=[tb_callback])


Train on 303 samples, validate on 101 samples
Epoch 1/20
303/303 [==============================] - 1s 3ms/sample - loss: 18.8849 - mean_absolute_error: 18.8849 - val_loss: 11.7991 - val_mean_absolute_error: 11.7991
Epoch 2/20
303/303 [==============================] - 1s 2ms/sample - loss: 6.8480 - mean_absolute_error: 6.8480 - val_loss: 4.7601 - val_mean_absolute_error: 4.7601
Epoch 3/20
303/303 [==============================] - 1s 2ms/sample - loss: 4.5795 - mean_absolute_error: 4.5795 - val_loss: 3.6579 - val_mean_absolute_error: 3.6579
Epoch 4/20
303/303 [==============================] - 1s 2ms/sample - loss: 3.8731 - mean_absolute_error: 3.8731 - val_loss: 3.2238 - val_mean_absolute_error: 3.2238
Epoch 5/20
303/303 [==============================] - 1s 2ms/sample - loss: 3.5502 - mean_absolute_error: 3.5502 - val_loss: 2.9367 - val_mean_absolute_error: 2.9367
Epoch 6/20
303/303 [==============================] - 1s 2ms/sample - loss: 3.2726 - mean_absolute_error: 3.2726 - val_l

In [0]:

mae_test=model.evaluate(test_data,test_targets)

102/102 [==============================] - 0s 620us/sample - loss: 2.7713 - mean_absolute_error: 2.7713


In [0]:
mae_test=model.evaluate(test_data,test_targets)
x_new = test_data[:3]
y_pred=model.predict(x_new)

102/102 [==============================] - 0s 95us/sample - loss: 2.7713 - mean_absolute_error: 2.7713


## RandomizedSearchCV

In [0]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [0]:
def create_model():
  return model

In [0]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(create_model)

In [0]:
keras_reg.fit(x_train,y_train,epochs=100,validation_data=(x_valid,y_valid),
              callbacks=[tb_callback]
             
             )

Train on 303 samples, validate on 101 samples
Epoch 1/100
303/303 [==============================] - 0s 974us/sample - loss: 2.2073 - mean_absolute_error: 2.2073 - val_loss: 2.1520 - val_mean_absolute_error: 2.1520
Epoch 2/100
303/303 [==============================] - 0s 148us/sample - loss: 2.1711 - mean_absolute_error: 2.1711 - val_loss: 2.1243 - val_mean_absolute_error: 2.1243
Epoch 3/100
303/303 [==============================] - 0s 148us/sample - loss: 2.1769 - mean_absolute_error: 2.1769 - val_loss: 2.1254 - val_mean_absolute_error: 2.1254
Epoch 4/100
303/303 [==============================] - 0s 132us/sample - loss: 2.1531 - mean_absolute_error: 2.1531 - val_loss: 2.1064 - val_mean_absolute_error: 2.1064
Epoch 5/100
303/303 [==============================] - 0s 115us/sample - loss: 2.1683 - mean_absolute_error: 2.1683 - val_loss: 2.1234 - val_mean_absolute_error: 2.1234
Epoch 6/100
303/303 [==============================] - 0s 115us/sample - loss: 2.1716 - mean_absolute_error: 

In [0]:
param_grid  = dict(epochs=[100], batch_size=[8])


In [0]:
cv = RandomizedSearchCV(keras_reg, param_grid,n_iter=10,cv=3)

In [0]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(create_model)

In [0]:
cv=RandomizedSearchCV(estimator = keras_reg, param_distributions=param_grid, cv = 2 )

In [0]:
#cv.fit(x_train,y_train)
             
cv.fit(x_train,y_train,epochs=100,validation_data=(x_valid,y_valid),callbacks=[tb_callback]
             )

Train on 151 samples, validate on 101 samples


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Epoch 1/100
151/151 [==============================] - 0s 2ms/sample - loss: 1.5971 - mean_absolute_error: 1.5971 - val_loss: 1.9836 - val_mean_absolute_error: 1.9836
Epoch 2/100
151/151 [==============================] - 0s 584us/sample - loss: 1.5467 - mean_absolute_error: 1.5467 - val_loss: 1.9978 - val_mean_absolute_error: 1.9978
Epoch 3/100
151/151 [==============================] - 0s 574us/sample - loss: 1.5800 - mean_absolute_error: 1.5800 - val_loss: 1.9590 - val_mean_absolute_error: 1.9590
Epoch 4/100
151/151 [==============================] - 0s 546us/sample - loss: 1.5109 - mean_absolute_error: 1.5109 - val_loss: 2.0095 - val_mean_absolute_error: 2.0095
Epoch 5/100
151/151 [==============================] - 0s 534us/sample - loss: 1.5824 - mean_absolute_error: 1.5824 - val_loss: 1.9297 - val_mean_absolute_error: 1.9297
Epoch 6/100
151/151 [==============================] - 0s 603us/sample - loss: 1.5357 - mean_absolute_error: 1.5357 - val_loss: 1.9313 - val_mean_absolute_er

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/100
303/303 [==============================] - 0s 1ms/sample - loss: 1.7451 - mean_absolute_error: 1.7451 - val_loss: 2.2482 - val_mean_absolute_error: 2.2482
Epoch 2/100
303/303 [==============================] - 0s 378us/sample - loss: 1.7087 - mean_absolute_error: 1.7087 - val_loss: 2.3525 - val_mean_absolute_error: 2.3525
Epoch 3/100
303/303 [==============================] - 0s 338us/sample - loss: 1.6960 - mean_absolute_error: 1.6960 - val_loss: 2.2882 - val_mean_absolute_error: 2.2882
Epoch 4/100
303/303 [==============================] - 0s 365us/sample - loss: 1.6807 - mean_absolute_error: 1.6807 - val_loss: 2.3192 - val_mean_absolute_error: 2.3192
Epoch 5/100
303/303 [==============================] - 0s 442us/sample - loss: 1.6658 - mean_absolute_error: 1.6658 - val_loss: 2.3173 - val_mean_absolute_error: 2.3173
Epoch 6/100
303/303 [==============================] - 0s 414us/sample - loss: 1.6475 - mean_absolute_error: 1.6475 - val_loss: 2.2700 - val_mean_absolute_er

RandomizedSearchCV(cv=2, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f5f9d541cc0>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'batch_size': [8], 'epochs': [100]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [0]:
cv.best_params_

{'batch_size': 8, 'epochs': 100}

In [0]:


cv.best_score_

-2.276891835058483